In [1]:
from engine import ModelEvaluator

evaluator = ModelEvaluator()
evaluator.evaluate()

/Users/artemsenin/Desktop/studies/vkr/quantized_nn/text_utils.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.meta["class"] = self.meta["class"].map(cls_map)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 